In [6]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
import os 

import multiprocessing
import threading

In [7]:
# musDB root dir
musdb_root_dir = '/home/scpark/hard/datasets/musdb18'

# directory to save the train data npz file
train_data_dir = '/home/scpark/hard/datasets/musdb18_wav'

# directory to save the test data npz file
test_data_dir = '/home/scpark/hard/datasets/musdb18_wav/test'

# chunk length (44100 sample rate, 10 seconds)
chunk_length = 10 * 44100

# the thread numbers to create data files
thread_nums = 4 # multiprocessing.cpu_count()

In [8]:
import musdb

# load the musDB
mus = musdb.DB(root_dir=musdb_root_dir)

# load the training tracks
train_tracks = mus.load_mus_tracks(subsets=['train'])

# load the test tracks
test_tracks = mus.load_mus_tracks(subsets=['test'])

In [9]:
def create_tracks_files(tracks, track_ids, save_dir):

    for track_id, track in zip(track_ids, tracks):

        length = len(track.audio)

        for j, start_index in tqdm(enumerate(range(0, length, chunk_length))):

            filename = save_dir + '/' + str(track_id) + '.' + str(j) + '.' + track.name + '.npz'

            # check if file already exist
            if os.path.exists(filename):
                continue

            # check if directory to save exist
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            audio = track.audio[start_index : start_index + chunk_length]
            vocals = track.targets['vocals'].audio[start_index : start_index + chunk_length]
            drums = track.targets['drums'].audio[start_index : start_index + chunk_length]
            bass = track.targets['bass'].audio[start_index : start_index + chunk_length]
            other = track.targets['other'].audio[start_index : start_index + chunk_length]
            accompaniment = track.targets['accompaniment'].audio[start_index : start_index + chunk_length]

            if len(audio) < chunk_length:
                pad_length = chunk_length - len(audio)
                audio = np.concatenate([audio, np.zeros([pad_length, 2])], axis=0)
                vocals = np.concatenate([vocals, np.zeros([pad_length, 2])], axis=0)
                drums = np.concatenate([drums, np.zeros([pad_length, 2])], axis=0)
                bass = np.concatenate([bass, np.zeros([pad_length, 2])], axis=0)
                other = np.concatenate([other, np.zeros([pad_length, 2])], axis=0)
                accompaniment = np.concatenate([accompaniment, np.zeros([pad_length, 2])], axis=0)

            elif len(audio) == 0:
                continue

            data = {
                    "audio": audio,
                    "vocals": vocals,
                    "drums": drums,
                    "bass": bass,
                    "other": other,
                    "accompaniment": accompaniment
                }

            np.savez(filename, **data, allow_pickle=False)

def create_files(tracks, save_dir, thread_nums):
    tracks_list = [[] for _ in range(thread_nums)]
    track_ids_list = [[] for _ in range(thread_nums)]
    
    for track_id, track in enumerate(tracks):
        tracks_list[track_id % thread_nums].append(track)
        track_ids_list[track_id % thread_nums].append(track_id)
        
        
    threads = []
    for thread_id in range(thread_nums):
        thread = threading.Thread(target=create_tracks_files, args=(tracks_list[thread_id], track_ids_list[thread_id], save_dir))
        thread.start()
        threads.append(thread)
        
    for thread in threads:
        thread.join()
    

In [10]:
create_files(train_tracks, train_data_dir, thread_nums)

In [5]:
create_files(test_tracks, test_data_dir, thread_nums)